# Introduction to Building LLM Agents with Tools and Tracing

This script walks through the process of building a simple LLM-powered agent that can use tools (functions) to answer questions. We'll cover:
1. Making basic LLM calls.
2. Introducing Weave for tracing and observability.
3. Defining tools for the LLM (manually and automatically).
4. Implementing a basic agentic loop.
5. Structuring the agent using Python classes.
6. Running the agent on a multi-step task.

**Prerequisites:**
Make sure you have the necessary libraries installed:
```bash
!pip install weave requests boto3
```


In [1]:
# Global Configuration & Setup
import inspect
import json
import os
import requests
import weave # Must import weave before litellm for auto-patching
from enum import Enum
from pydantic import BaseModel, Field
from rich.pretty import pprint
from typing import Any, Callable, Dict, List, get_type_hints

In [2]:
# you need to request access to the model on the Bedrock UI
CLAUDE_MODEL = "us.anthropic.claude-sonnet-4-20250514-v1:0"

Define a model to use, as we are going to use tool calling you need a capable model like `claude-4`

Let's log to [W&B Weave](https://weave-docs.wandb.ai/). Weights & Biases (W&B) Weave is a framework for tracking, experimenting with, evaluating, deploying, and improving LLM-based applications. Designed for flexibility and scalability, Weave supports every stage of your LLM application development workflow:

- Tracing & Monitoring: Track LLM calls and application logic to debug and analyze production systems.
- Systematic Iteration: Refine and iterate on prompts, datasets, and models.
- Experimentation: Experiment with different models and prompts in the LLM Playground.
- Evaluation: Use custom or pre-built scorers alongside our comparison tools to systematically assess and enhance application performance.
- Guardrails: Protect your application with pre- and post-safeguards for content moderation, prompt safety, and more.

In [3]:

# Initialize a Weave project. Traces will be sent here.
# You can view them in the Weave UI (usually runs locally).
weave.init('reliable-agents')

weave version 0.51.54 is available!  To upgrade, please run:
 $ pip install weave --upgrade
Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/reliable-agents/weave


## 1. Basic LLM Call with Bedrock Converse API 

Let's start with a simple call to the LLM using/

![](br_images/01_traces.png)

In [4]:
import boto3
from weave.integrations.bedrock.bedrock_sdk import patch_client

# Create and patch the Bedrock client
client = boto3.client("bedrock-runtime")
patch_client(client)

messages = [{"role": "user", "content": [{"text": "Hello, LLM! How does an AI agent work?"}]}]

# call the model using the converse API
response = client.converse(
    modelId=CLAUDE_MODEL,
    system=[{"text": "You are a helpful AI assistant."}],
    messages=messages,
    inferenceConfig={"maxTokens": 400},
)
pprint(response)

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abcd-0833-7f63-85ba-1c62ed3abda6


{
│   'ResponseMetadata': {
│   │   'RequestId': 'fb6193eb-6bc2-4166-a740-6d1e9cd0bf81',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Thu, 26 Jun 2025 10:33:59 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '1794',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'fb6193eb-6bc2-4166-a740-6d1e9cd0bf81'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {
│   │   │   │   │   'text': "Hello! Great question! An AI agent is a system designed to perceive its environment, make decisions, and take actions to achieve specific goals. Here's how they typically work:\n\n## Core Components:\n\n**1. Perception**\n- Sensors or input mechanisms to gather information from the environment\n- Could be cameras, microphones, text inputs, data feeds, etc.\n\n**2. Decision-Making Engine**\n- Processing unit (often neural networks, rule-based systems, or hybrid approaches)\n- Evaluates current state and determines best actions\n- May use techniques like machine learning, reinforcement learning, or classical AI\n\n**3. Action Mechanism**\n- Ways to interact with and modify the environment\n- Could be physical actuators, software commands, generating text/images, etc.\n\n## Basic Cycle:\n1. **Observe** the current state of the environment\n2. **Process** that information using its knowledge/training\n3. **Decide** on the best action to take\n4. **Act** to influence the environment\n5. **Learn** from the results (in adaptive systems)\n6. Repeat the cycle\n\n## Types of AI Agents:\n- **Reactive**: Respond directly to current inputs\n- **Deliberative**: Plan ahead using internal models\n- **Learning**: Improve performance over time\n- **Multi-agent**: Multiple agents working together\n\nThe sophistication can range from simple rule-following bots to complex systems like autonomous vehicles or conversational AI like myself!\n\nWould you like me to elaborate on any particular aspect?"
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 27,
│   │   'outputTokens': 338,
│   │   'totalTokens': 365,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 7744}
}

To get your LLM calls traces, you need to call `patch_client(client)` on the `boto3` Bedrock Runtime client. You can follow the link to your Weave dashboard and see the trace, including the input messages, output response, latency, model used, etc. This is invaluable for debugging and monitoring.

In [ ]:
# most of the time you would want to define your own operations to trace, for instance to call the model.
# You just need to add the @weave.op decorator to the function and it will be traced.

@weave.op
def call_model(model_name: str, messages: List[Dict[str, Any]], system: List[Dict[str, Any]]=[{"text": "You are a helpful AI assistant."}], **kwargs) -> str:
    "Call a model with the given messages and kwargs."

    # call the model using the converse API
    response = client.converse(
        modelId=CLAUDE_MODEL,
        system=system,
        messages=messages,
        inferenceConfig={"maxTokens": 400},
        **kwargs
    )
    return response

response = call_model(model_name=CLAUDE_MODEL, messages=messages)
pprint(response)

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abcd-f2f4-7e32-8d24-b2607fbef161


{
│   'ResponseMetadata': {
│   │   'RequestId': 'd1c6c497-e005-4701-b60f-abce4b70615d',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Thu, 26 Jun 2025 10:34:52 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '420',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'd1c6c497-e005-4701-b60f-abce4b70615d'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {
│   │   │   │   │   'text': "To find the sum of your lucky numbers 77 and 11, I'll add them together:\n\n77 + 11 = 88\n\nThe sum of your lucky numbers is 88."
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 30,
│   │   'outputTokens': 45,
│   │   'totalTokens': 75,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 1435}
}

![](br_images/02_nested_tracing.png)


## 2. Introducing Tool Calling

Agents become much more powerful when they can use **tools** – external functions or APIs – to get information or perform actions beyond the LLM's internal knowledge. To allow an LLM to use a tool, we need to provide it with a description (schema) of the tool, including its name, purpose, and expected arguments.

Check the Mistral docs for function calling: https://docs.mistral.ai/capabilities/function_calling/

![](br_images/03_tools.png)

First, let's define a simple Python function we want the LLM to be able to call. We add `@weave.op` to trace when this function actually gets executed.


In [12]:
@weave.op 
def add_numbers(a: int, b: int) -> int:
    """Adds two numbers.
    Args:
        a: The first number.
        b: The second number.
    """
    return a + b

In [13]:
add_numbers(1, 2)

3

In [14]:
# this doesn't work...
call_model(model_name=CLAUDE_MODEL, messages=messages, tools=[add_numbers])

🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abce-143c-7ee0-8886-2eac373a69f3
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abd4-8ab7-7c41-8b10-27fd7c6d782d
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abda-1845-7080-b146-2541558972fb
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abe0-be87-7c93-bb70-f4589be54a12
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abe2-fbda-7560-9da1-0324d7126a3d
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abe4-a847-7070-a22c-a2865c58dc8b
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abf0-a159-7e80-a7b6-f6428c342b34
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abf0-e460-7b03-af52-6d2930bdd9a6
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abf4-f417-79d1-a66b-9fdb254a26d3
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abf6-f198-7241-834c-7cbf968644c1
🍩 https://wandb.ai/capecape/reliable-agents/r/call/0197abf7-f69a-72c2-b4b8-2e2da0241d19
🍩 https://wandb.ai/capecape/reli

ParamValidationError: Parameter validation failed:
Unknown parameter in input: "tools", must be one of: modelId, messages, system, inferenceConfig, toolConfig, guardrailConfig, additionalModelRequestFields, promptVariables, additionalModelResponseFieldPaths, requestMetadata, performanceConfig

> We need to manually create the JSON schema describing this tool in a format that models *Mistral* understand.

In [15]:
# Manually define the tool schema
tool_add_numbers_schema = {
  "toolSpec": {
    "name": "add_numbers",
    "description": "Adds two numbers.",
    "inputSchema": {
      "json": {
        "type": "object",
        "properties": {
          "a": {
            "type": "integer",
            "description": "The first number."
          },
          "b": {
            "type": "integer",
            "description": "The second number."
          }
        },
        "required": ["a", "b"]
      }
    }
  }
}


tool_config = { "tools": [tool_add_numbers_schema] }

Now, we make an LLM call, passing the `tools` parameter with our schema. We ask a question that should trigger the tool.

In [65]:
system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
messages = [{"role": "user", "content": [{"text": "My lucky numbers are 77 and 11. What is their sum?"}]}]
response = call_model(model_name=CLAUDE_MODEL, messages=messages, system=system, toolConfig=tool_config)
pprint(response)

{
│   'ResponseMetadata': {
│   │   'RequestId': 'cbd7cc0f-63de-45bf-ac98-f34e9fc4f13e',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Thu, 26 Jun 2025 10:59:34 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '457',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'cbd7cc0f-63de-45bf-ac98-f34e9fc4f13e'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': "I'll help you find the sum of your lucky numbers 77 and 11."},
│   │   │   │   {
│   │   │   │   │   'toolUse': {
│   │   │   │   │   │   'toolUseId': 'tooluse_7JFqX4scTWuIXPtc3EmoNQ',
│   │   │   │   │   │   'name': 'add_numbers',
│   │   │   │   │   │   'input': {'a': 77, 'b': 11}
│   │   │   │   │   }
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'tool_use',
│   'usage': {
│   │   'inputTokens': 431,
│   │   'outputTokens': 89,
│   │   'totalTokens': 520,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 2003}
}

## Manual Tool Call
The LLM's response might contain a request to call our tool. If it requests a tool call, we need to:

1. Parse the arguments it provides.
2. Execute our actual Python function (`add_numbers`) with those arguments.
3. (In a real agent loop) Send the result back to the LLM in a new message with `role="tool"`.

Let's manually call the tools in the response.

In [66]:
if response["stopReason"] == "tool_use":
    print("LLM requested a tool call:")
    for message in response["output"]["message"]["content"]:
        if message.get("text"):
            print(message["text"])
        if message.get("toolUse", False):
            tool_name = message["toolUse"]["name"]
            tool_args = message["toolUse"]["input"]
            tool_use_id = message["toolUse"]["toolUseId"]
            print(f"  - Tool: {tool_name}, Args: {tool_args}")
            if tool_name == "add_numbers":
                tool_result_content = add_numbers(**tool_args)

print(tool_result_content)

LLM requested a tool call:
I'll help you find the sum of your lucky numbers 77 and 11.
  - Tool: add_numbers, Args: {'a': 77, 'b': 11}
88


We need to add the tool call result to the messages (there is actually 2 messages to add)
- the response from the assistant that decided to call the tool
- the tool output

In [67]:
messages.append(response["output"]["message"])

In [68]:
messages

[{'role': 'user',
  'content': [{'text': 'My lucky numbers are 77 and 11. What is their sum?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find the sum of your lucky numbers 77 and 11."},
   {'toolUse': {'toolUseId': 'tooluse_7JFqX4scTWuIXPtc3EmoNQ',
     'name': 'add_numbers',
     'input': {'a': 77, 'b': 11}}}]}]

In [69]:
tool_result = {
        "toolUseId": tool_use_id,
        "status": "success",
        "content": [{"json": {"sum": tool_result_content}}]
}

In [70]:
messages.append({"role": "user", "content": [{"toolResult": tool_result}]})

In [71]:
messages

[{'role': 'user',
  'content': [{'text': 'My lucky numbers are 77 and 11. What is their sum?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you find the sum of your lucky numbers 77 and 11."},
   {'toolUse': {'toolUseId': 'tooluse_7JFqX4scTWuIXPtc3EmoNQ',
     'name': 'add_numbers',
     'input': {'a': 77, 'b': 11}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_7JFqX4scTWuIXPtc3EmoNQ',
     'status': 'success',
     'content': [{'json': {'sum': 88}}]}}]}]

Now call the model again with the new messages and it will use the tool call result to answer the question

In [75]:
final_response = call_model(model_name=CLAUDE_MODEL, messages=messages, system=system, toolConfig=tool_config)
pprint(final_response)

{
│   'ResponseMetadata': {
│   │   'RequestId': 'fdcfdb26-eaaa-4c1f-acd7-68d499f12c0b',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Thu, 26 Jun 2025 11:00:07 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '340',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'fdcfdb26-eaaa-4c1f-acd7-68d499f12c0b'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {'role': 'assistant', 'content': [{'text': 'The sum of your lucky numbers 77 and 11 is 88.'}]}
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 536,
│   │   'outputTokens': 20,
│   │   'totalTokens': 556,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 1361}
}

## 3. Simplifying Tool Definition with a Processor Function

Manually writing JSON schemas is tedious and error-prone. We can automate this by inspecting our Python function's signature, type hints, and docstring.

First, let's define a helper function (`generate_tool_schema`) that takes a Python function and generates the schema.


In [78]:
def generate_tool_schema(func: Callable) -> dict:
    """Given a Python function, generate a Bedrock-compatible JSON schema.
    Handles basic types and Enums. Assumes docstrings are formatted for arg descriptions.
    """
    signature = inspect.signature(func)
    parameters = signature.parameters
    type_hints = get_type_hints(func)

    # Extract main description (everything before Args: section)
    docstring = inspect.getdoc(func)
    main_description = ""
    if docstring:
        lines = docstring.split('\n')
        for line in lines:
            if line.strip().lower().startswith(("args:", "arguments:", "parameters:")):
                break
            if main_description:
                main_description += " "
            main_description += line.strip()

    schema = {
        "toolSpec": {
            "name": func.__name__,
            "description": main_description,
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {},
                    "required": [],
                }
            }
        }
    }

    # Parse parameter descriptions from docstring
    param_descriptions = {}
    if docstring:
        args_section = False
        for line in docstring.split('\n'):
            line_stripped = line.strip()
            if line_stripped.lower().startswith(("args:", "arguments:", "parameters:")):
                args_section = True
                continue
            if args_section:
                if ":" in line_stripped and line_stripped[0] != ' ':
                    param_name, desc = line_stripped.split(":", 1)
                    param_descriptions[param_name.strip()] = desc.strip()
                elif line_stripped and not line_stripped.startswith(" ") and ":" not in line_stripped:
                    # End of args section
                    args_section = False

    for name, param in parameters.items():
        is_required = param.default == inspect.Parameter.empty
        param_type = type_hints.get(name, Any)
        json_type = "string"
        param_schema = {}

        # Basic type mapping
        if param_type == str: json_type = "string"
        elif param_type == int: json_type = "integer"
        elif param_type == float: json_type = "number"
        elif param_type == bool: json_type = "boolean"
        elif hasattr(param_type, '__origin__') and param_type.__origin__ is list: # Handle List[type]
             item_type = param_type.__args__[0] if param_type.__args__ else Any
             if item_type == str: param_schema = {"type": "array", "items": {"type": "string"}}
             elif item_type == int: param_schema = {"type": "array", "items": {"type": "integer"}}
             # Add more list item types if needed
             else: param_schema = {"type": "array", "items": {"type": "string"}} # Default list item type
        elif hasattr(param_type, "__members__") and issubclass(param_type, Enum): # Handle Enum
             json_type = "string"
             param_schema["enum"] = [e.value for e in param_type]

        if not param_schema: # If not set by List or Enum
            param_schema["type"] = json_type

        param_schema["description"] = param_descriptions.get(name, "")

        if param.default != inspect.Parameter.empty and param.default is not None:
             param_schema["default"] = param.default # Note: Bedrock schema doesn't officially use default, but useful metadata

        schema["toolSpec"]["inputSchema"]["json"]["properties"][name] = param_schema
        if is_required:
            schema["toolSpec"]["inputSchema"]["json"]["required"].append(name)
    return schema

Now we can use this function to automatically generate the schema for our tool.

In [79]:
tool_schema = generate_tool_schema(add_numbers)
pprint(tool_schema)

{
│   'toolSpec': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.',
│   │   'inputSchema': {
│   │   │   'json': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'a': {'type': 'integer', 'description': 'The first number.'},
│   │   │   │   │   'b': {'type': 'integer', 'description': 'The second number.'}
│   │   │   │   },
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   }
}

Now, we define a `function_tool` "processor". This isn't a decorator in the `@` syntax sense here, but a function that we call *after* defining our tool function. It uses `generate_tool_schema` to attach the schema to the function object itself.


In [80]:
def function_tool(func: Callable) -> Callable:
    """Attaches a tool schema to the function and marks it as a tool.
    Call this *after* defining your function: my_func = function_tool(my_func)
    """
    try:
        func.tool_schema = generate_tool_schema(func)
        func.is_tool = True # Mark it as a tool
    except Exception as e:
        print(f"Error processing tool {func.__name__}: {e}")
        # Optionally raise or mark as failed
        func.tool_schema = None
        func.is_tool = False
    return func

We can use this function to automatically generate the schema for our tool, as a decorator or after the function is defined.

In [81]:
add_numbers = function_tool(add_numbers)
pprint(add_numbers.tool_schema)


{
│   'toolSpec': {
│   │   'name': 'add_numbers',
│   │   'description': 'Adds two numbers.',
│   │   'inputSchema': {
│   │   │   'json': {
│   │   │   │   'type': 'object',
│   │   │   │   'properties': {
│   │   │   │   │   'a': {'type': 'integer', 'description': 'The first number.'},
│   │   │   │   │   'b': {'type': 'integer', 'description': 'The second number.'}
│   │   │   │   },
│   │   │   │   'required': ['a', 'b']
│   │   │   }
│   │   }
│   }
}

In [82]:
add_numbers.tool_schema

{'toolSpec': {'name': 'add_numbers',
  'description': 'Adds two numbers.',
  'inputSchema': {'json': {'type': 'object',
    'properties': {'a': {'type': 'integer',
      'description': 'The first number.'},
     'b': {'type': 'integer', 'description': 'The second number.'}},
    'required': ['a', 'b']}}}}

and call the tool =)

In [83]:
add_numbers(1, 2)

3

### 3.1 Real Example using an API based tool

Let's create a real example hitting an API: The pokemon API

In [84]:
@weave.op 
@function_tool # <- we can use the decorator to automatically generate the tool schema
def get_pokemon_info(pokemon_name: str) -> str:
    """Fetches minimal information (name, ID, weight) for a specific Pokemon. Weight is in hectograms.

    Args:
        pokemon_name: The name or Pokedex ID of the Pokemon.
    """
    base_url = "https://pokeapi.co/api/v2/pokemon/"
    try:
        response = requests.get(f"{base_url}{pokemon_name.lower().strip()}")
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        info = {
            "name": data.get('name', 'Unknown').capitalize(),
            "id": data.get('id', -1),
            "weight": data.get('weight', -1) # Weight in hectograms
        }
        return info
    except Exception as e:
        return {"error": f"Error fetching pokemon {pokemon_name}: {str(e)}"}

In [85]:
get_pokemon_info.tool_schema

{'toolSpec': {'name': 'get_pokemon_info',
  'description': 'Fetches minimal information (name, ID, weight) for a specific Pokemon. Weight is in hectograms. ',
  'inputSchema': {'json': {'type': 'object',
    'properties': {'pokemon_name': {'type': 'string',
      'description': 'The name or Pokedex ID of the Pokemon.'}},
    'required': ['pokemon_name']}}}}

In [86]:
get_pokemon_info("pikachu")

{'name': 'Pikachu', 'id': 25, 'weight': 60}

In [151]:
system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
messages = [{"role": "user", "content": [{"text":"What is the weight of Pikachu?"}]}]
tool_config = { "tools": [get_pokemon_info.tool_schema] }

response = call_model(model_name=CLAUDE_MODEL, messages=messages, system=system, toolConfig=tool_config)
pprint(response)

{
│   'ResponseMetadata': {
│   │   'RequestId': 'd2f2fb70-48a0-49f2-bfcb-08c7609e85b7',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Thu, 26 Jun 2025 11:33:36 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '452',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'd2f2fb70-48a0-49f2-bfcb-08c7609e85b7'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {'text': "I'll look up Pikachu's weight for you."},
│   │   │   │   {
│   │   │   │   │   'toolUse': {
│   │   │   │   │   │   'toolUseId': 'tooluse_TkWI6RUgSzaI3JRls2gKKQ',
│   │   │   │   │   │   'name': 'get_pokemon_info',
│   │   │   │   │   │   'input': {'pokemon_name': 'Pikachu'}
│   │   │   │   │   }
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'tool_use',
│   'usage': {
│   │   'inputTokens': 438,
│   │   'outputTokens': 74,
│   │   'totalTokens': 512,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 1980}
}

Let's create some helper functions to perform the tool calls

In [152]:
def get_tool(tools: list[Callable], name: str) -> Callable:
    for t in tools:
        if t.__name__ == name:
            return t
    raise KeyError(f"No tool with name {name} found")

def perform_tool_calls(tools: list[Callable], response: dict) -> list[dict]:
    """Perform the tool calls from a Bedrock response and return the messages with the tool call results"""
    tool_content = []
    if response["stopReason"] == "tool_use":
        print("LLM requested tool calls:")
        for message in response["output"]["message"]["content"]:
            if message.get("text"):
                print(message["text"])
            if message.get("toolUse", False):
                tool_name = message["toolUse"]["name"]
                tool_args = message["toolUse"]["input"]
                tool_use_id = message["toolUse"]["toolUseId"]
                print(f"  - Tool: {tool_name}, Args: {tool_args}")
                
                try:
                    # Get the tool function and call it
                    tool = get_tool(tools, tool_name)
                    tool_response = tool(**tool_args)
                    print(f"  - Response: {tool_response}")
                    
                    # Format the tool result for Bedrock
                    tool_result = {
                        "toolUseId": tool_use_id,
                        "status": "success",
                        "content": [{"json": {"result": tool_response}}]
                    }
                    
                    # Add the tool result message
                    tool_content.append({"toolResult": tool_result})
                    
                except Exception as e:
                    print(f"  - Error: {e}")
                    # Format error response
                    tool_result = {
                        "toolUseId": tool_use_id,
                        "status": "error",
                        "content": [{"text": f"Error executing tool: {str(e)}"}]
                    }
                    tool_content.append({"toolResult": tool_result})


    return {"role": "user", "content": tool_content}

In [153]:

# add the tool call result to the messages
messages.append(response["output"]["message"])

# add the tool call result to the messages
messages.append(perform_tool_calls(tools=[get_pokemon_info], response=response))

LLM requested tool calls:
I'll look up Pikachu's weight for you.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'Pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}


In [154]:
messages

[{'role': 'user', 'content': [{'text': 'What is the weight of Pikachu?'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll look up Pikachu's weight for you."},
   {'toolUse': {'toolUseId': 'tooluse_TkWI6RUgSzaI3JRls2gKKQ',
     'name': 'get_pokemon_info',
     'input': {'pokemon_name': 'Pikachu'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_TkWI6RUgSzaI3JRls2gKKQ',
     'status': 'success',
     'content': [{'json': {'result': {'name': 'Pikachu',
         'id': 25,
         'weight': 60}}}]}}]}]

In [155]:
final_response = call_model(model_name=CLAUDE_MODEL, messages=messages, toolConfig=tool_config)
pprint(final_response)


{
│   'ResponseMetadata': {
│   │   'RequestId': 'bd6318a7-ad77-40b0-9e2d-5634148cbd3d',
│   │   'HTTPStatusCode': 200,
│   │   'HTTPHeaders': {
│   │   │   'date': 'Thu, 26 Jun 2025 11:33:40 GMT',
│   │   │   'content-type': 'application/json',
│   │   │   'content-length': '380',
│   │   │   'connection': 'keep-alive',
│   │   │   'x-amzn-requestid': 'bd6318a7-ad77-40b0-9e2d-5634148cbd3d'
│   │   },
│   │   'RetryAttempts': 0
│   },
│   'output': {
│   │   'message': {
│   │   │   'role': 'assistant',
│   │   │   'content': [
│   │   │   │   {
│   │   │   │   │   'text': 'Pikachu weighs 60 hectograms, which is equivalent to 6 kilograms or about 13.2 pounds.'
│   │   │   │   }
│   │   │   ]
│   │   }
│   },
│   'stopReason': 'end_turn',
│   'usage': {
│   │   'inputTokens': 537,
│   │   'outputTokens': 35,
│   │   'totalTokens': 572,
│   │   'cacheReadInputTokens': 0,
│   │   'cacheWriteInputTokens': 0
│   },
│   'metrics': {'latencyMs': 1563}
}

Let's wrap this in a function and add a few more tools.

In [166]:
@weave.op
def pokedex(pokemon_question: str) -> str:
    system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
    messages = [{"role": "user", "content": [{"text": pokemon_question}]}]
    
    # Create tool config for Bedrock
    tool_config = {"tools": [get_pokemon_info.tool_schema]}

    # call model with tools
    response = call_model(
        model_name=CLAUDE_MODEL, 
        messages=messages,
        system=system,
        toolConfig=tool_config
    )

    # add the assistant's response to the messages
    messages.append(response["output"]["message"])
    # pprint(messages)

    # perform the tool calls if any
    tool_result_messages = perform_tool_calls(tools=[get_pokemon_info], response=response)
    messages.append(tool_result_messages)
    # pprint(messages)

    # get final response
    final_response = call_model(
        model_name=CLAUDE_MODEL, 
        messages=messages,
        system=system,
        toolConfig=tool_config
    )
    
    return final_response["output"]["message"]["content"][-1]["text"]

In [168]:
response = pokedex("What is the combined weight of Ash's first 3 pokemons?")
print(response)

LLM requested tool calls:
I'll help you find the combined weight of Ash's first 3 Pokémon. Ash's original team consisted of Pikachu, Bulbasaur, and Charmander. Let me get their weight information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'bulbasaur'}
  - Response: {'name': 'Bulbasaur', 'id': 1, 'weight': 69}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'charmander'}
  - Response: {'name': 'Charmander', 'id': 4, 'weight': 85}
Based on the information retrieved:

- **Pikachu**: 60 hectograms (6.0 kg)
- **Bulbasaur**: 69 hectograms (6.9 kg) 
- **Charmander**: 85 hectograms (8.5 kg)

**Combined weight**: 60 + 69 + 85 = **214 hectograms** (21.4 kg or 47.2 lbs)

So Ash's first three Pokémon have a combined weight of 214 hectograms or 21.4 kilograms.


![](br_images/04_pokedex.png)

This is "Almost" an agent, but it's missing the loop. Let's add that next.

## 4. Implementing a Basic Agentic Loop

Let's implement a basic agentic loop. We'll use the `pokedex` function we just created. The implementation we have above has some limitations:
- Its a single turn, so if it fails to answer my question in one pass it is over.

![](images/05_agent.png)

From the really good [Anthropic Building Effective Agents](https://www.anthropic.com/engineering/building-effective-agents) article and encourage people to read it.

A simple for loop

In [169]:
@weave.op
def pokedex_loop(pokemon_question: str, max_turns: int = 4, tools = [get_pokemon_info, add_numbers]) -> str:
    system = [{"text": "You are a helpful assistant that can use tools to answer questions."}]
    messages = [{"role": "user", "content": [{"text": pokemon_question}]}]
    
    # Create tool config for Bedrock
    tool_config = {"tools": [t.tool_schema for t in tools]}
    
    for turn in range(max_turns):
        print(f"--- Agent Loop Turn {turn + 1}/{max_turns} ---")

        # call model with tools
        response = call_model(
            model_name=CLAUDE_MODEL, 
            messages=messages,
            system=system,
            toolConfig=tool_config
        )

        # add the assistant's response to the messages
        messages.append(response["output"]["message"])

        # if the LLM requested tool calls, perform them
        if response["stopReason"] == "tool_use":
            print("LLM requested tool calls:")
            # perform the tool calls
            tool_result_messages = perform_tool_calls(tools=tools, response=response)
            messages.append(tool_result_messages)
        # LLM gave content response
        elif response["stopReason"] == "end_turn":
            # Extract text content from response
            content = ""
            for content_item in response["output"]["message"]["content"]:
                if content_item.get("text"):
                    content += content_item["text"]
            print(f"LLM content response: {content}")
            return content
        else:
            print(f"LLM response had unexpected stop reason: {response['stopReason']}. Stopping loop.")
            break
    
    # If we reach max_turns without a final response, return the last assistant message
    if messages and messages[-1]["role"] == "assistant":
        content = ""
        for content_item in messages[-1]["content"]:
            if content_item.get("text"):
                content += content_item["text"]
        return content
    
    return "No response generated within max turns."

In [170]:
pokedex_loop("What is the combined weight of Ash's first 3 pokemons?")

--- Agent Loop Turn 1/4 ---
LLM requested tool calls:
LLM requested tool calls:
I'll help you find the combined weight of Ash's first 3 Pokémon. Ash's first three Pokémon were Pikachu, Caterpie, and Pidgeotto. Let me get their weight information.
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'caterpie'}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pidgeotto'}
  - Response: {'name': 'Pidgeotto', 'id': 17, 'weight': 300}
--- Agent Loop Turn 2/4 ---
LLM requested tool calls:
LLM requested tool calls:
Now let me add up their weights:
  - Tool: add_numbers, Args: {'a': 60, 'b': 29}
  - Response: 89
--- Agent Loop Turn 3/4 ---
LLM requested tool calls:
LLM requested tool calls:
  - Tool: add_numbers, Args: {'a': 89, 'b': 300}
  - Response: 389
--- Agent Loop Turn 4/4 ---
LLM content response: The combine

"The combined weight of Ash's first 3 Pokémon is **389 hectograms** (which equals 38.9 kg or about 85.8 lbs).\n\nHere's the breakdown:\n- Pikachu: 60 hectograms\n- Caterpie: 29 hectograms  \n- Pidgeotto: 300 hectograms\n- **Total: 389 hectograms**"

![](br_images/05_pokedex_loop.png)

This one is kind of an hallucination, I was expecting snorlax to be the heaviest pokemon.

In [172]:
pokedex_loop("Which is the heaviest pokemon?")

--- Agent Loop Turn 1/4 ---
LLM content response: I'd be happy to help you find the heaviest Pokémon! However, I don't have access to a comprehensive database or comparison tool that would let me search through all Pokémon at once to find the heaviest one.

With the tools I have available, I can only look up individual Pokémon by name or ID to get their weight information. If you have some specific Pokémon in mind that you think might be among the heaviest (like some of the legendary or especially large Pokémon), I could look those up and compare their weights for you.

Some Pokémon that are often considered among the heaviest include:
- Groudon
- Dialga
- Giratina
- Wailord
- Celesteela

Would you like me to check the weights of any specific Pokémon, or do you have particular ones in mind that you'd like me to compare?


"I'd be happy to help you find the heaviest Pokémon! However, I don't have access to a comprehensive database or comparison tool that would let me search through all Pokémon at once to find the heaviest one.\n\nWith the tools I have available, I can only look up individual Pokémon by name or ID to get their weight information. If you have some specific Pokémon in mind that you think might be among the heaviest (like some of the legendary or especially large Pokémon), I could look those up and compare their weights for you.\n\nSome Pokémon that are often considered among the heaviest include:\n- Groudon\n- Dialga\n- Giratina\n- Wailord\n- Celesteela\n\nWould you like me to check the weights of any specific Pokémon, or do you have particular ones in mind that you'd like me to compare?"

## 5. Structuring the Agent with Classes

The loop above works, but for more complex agents, encapsulating the logic and state within classes is much better. We'll define:
- `AgentState`: A Pydantic model to hold the conversation history and potentially other state.
- `SimpleAgent`: A class containing the agent's configuration (model, system message, tools) and logic (`step`, `run`).

In [173]:
class AgentState(BaseModel):
    """Manages the state of the agent."""
    messages: List[Dict[str, Any]] = Field(default_factory=list)
    step: int = Field(default=0)
    final_assistant_content: str | None = None # Populated at the end of a run

class SimpleAgent:
    """A simple agent class with tracing, state, and tool processing."""
    def __init__(self, model_name: str, system_message: str, tools: List[Callable]):
        self.model_name = model_name
        self.system_message = system_message
        self.tools = [function_tool(t) for t in tools] # add schemas to the tools
    
    @weave.op(name="SimpleAgent.step") # Trace each step
    def step(self, state: AgentState) -> AgentState:
        step = state.step + 1
        messages = state.messages
        final_assistant_content = None
        
        try:
            # Create system and tool config for Bedrock
            system = [{"text": self.system_message}]
            tool_config = {"tools": [t.tool_schema for t in self.tools]}
            
            # call model with tools
            response = call_model(
                model_name=self.model_name, 
                messages=messages,
                system=system,
                toolConfig=tool_config
            )

            # add the assistant's response to the messages
            messages.append(response["output"]["message"])

            # if the LLM requested tool calls, perform them
            if response["stopReason"] == "tool_use":
                print("LLM requested tool calls:")
                # perform the tool calls
                tool_result_messages = perform_tool_calls(tools=self.tools, response=response)
                messages.append(tool_result_messages)
            
            # LLM gave content response
            elif response["stopReason"] == "end_turn":
                # Extract text content from Bedrock response
                content = ""
                for content_item in response["output"]["message"]["content"]:
                    if content_item.get("text"):
                        content += content_item["text"]
                final_assistant_content = content
            else:
                print(f"Unexpected stop reason: {response['stopReason']}")
                
        except Exception as e:
            print(f"ERROR in Agent Step: {e}")
            # Add an error message to history in Bedrock format
            error_message = {
                "role": "assistant", 
                "content": [{"text": f"Agent error in step: {str(e)}"}]
            }
            messages.append(error_message)
            final_assistant_content = f"Agent error in step {step}: {str(e)}"
            
        return AgentState(messages=messages, step=step, final_assistant_content=final_assistant_content)

    @weave.op(name="SimpleAgent.run")
    def run(self, user_prompt: str, max_turns: int = 10) -> AgentState:
        # Initialize state with Bedrock message format
        state = AgentState(messages=[
            {"role": "user", "content": [{"text": user_prompt}]}
        ])
        
        for _ in range(max_turns):
            print(f"--- Agent Loop Turn {state.step + 1}/{max_turns} ---")
            state = self.step(state)
            if state.final_assistant_content:
                return state
        return state

![](images/07_simple_agent.png)

In [174]:
agent = SimpleAgent(
    model_name=CLAUDE_MODEL,
    system_message="You are a helpful assistant that can use tools to answer questions.",
    tools=[get_pokemon_info, add_numbers]
)
state = agent.run(user_prompt="What is the combined weight of Ash's first 3 pokemons?")
print(f"Final response: {state.final_assistant_content}")


--- Agent Loop Turn 1/10 ---
LLM requested tool calls:
LLM requested tool calls:
I'll help you find the combined weight of Ash's first 3 Pokémon. Ash's first three Pokémon were Pikachu, Caterpie, and Pidgeotto. Let me get their weight information:
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pikachu'}
  - Response: {'name': 'Pikachu', 'id': 25, 'weight': 60}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'caterpie'}
  - Response: {'name': 'Caterpie', 'id': 10, 'weight': 29}
  - Tool: get_pokemon_info, Args: {'pokemon_name': 'pidgeotto'}
  - Response: {'name': 'Pidgeotto', 'id': 17, 'weight': 300}
--- Agent Loop Turn 2/10 ---
LLM requested tool calls:
LLM requested tool calls:
Now let me calculate the combined weight:
  - Tool: add_numbers, Args: {'a': 60, 'b': 29}
  - Response: 89
--- Agent Loop Turn 3/10 ---
LLM requested tool calls:
LLM requested tool calls:
  - Tool: add_numbers, Args: {'a': 89, 'b': 300}
  - Response: 389
--- Agent Loop Turn 4/10 ---
Final response: The 

![](br_images/06_agent.png)

Possible improvements to the SimpleAgent:
- Give the model info about the state of the conversation, you could inject a message with the model context pressure, steps left, etc.
- Structured output. Make the model output a specific format, for instance a JSON with the expected fields.
- Add more tools like read and write files, access a database.
- Agent handoff: Agent1 does triage and Agent2 executes specific tasks.